In [19]:
from pathlib import Path
import sys

root = str(Path().resolve().parents[0])
if root not in sys.path:
    sys.path.append(root)

from src.utils import ROOT, DATA_DIR, EXPERIMENTS_DIR, OUTPUT_DIR, DATA_PATHS

for path in [ROOT, DATA_DIR, EXPERIMENTS_DIR, OUTPUT_DIR]:
    if str(path.resolve()) not in sys.path:
        sys.path.append(str(path.resolve()))

print(*sys.path, sep="\n")

/usr/lib/python312.zip
/usr/lib/python3.12
/usr/lib/python3.12/lib-dynload

/home/ryanwtsai/repos/ml_ai_portfolio/notes/example_code/venv/lib/python3.12/site-packages
/home/ryanwtsai/repos/ml_ai_portfolio/titanic
/home/ryanwtsai/repos/ml_ai_portfolio/titanic/dataset
/home/ryanwtsai/repos/ml_ai_portfolio/titanic/experiments
/home/ryanwtsai/repos/ml_ai_portfolio/titanic/outputs


In [22]:
from importlib import reload
import src
# import experiment_config

reload(src)
reload(src.utils)
reload(src.custom_transformers)
reload(src.custom_model_selection)
reload(src.experiment_runner)
reload(src.config_parser)
# reload(experiment_config)

<module 'src.config_parser' from '/home/ryanwtsai/repos/ml_ai_portfolio/titanic/src/config_parser.py'>

In [29]:
import yaml
from src.config import build_estimators

with open(ROOT / "scripts/experiment_config.yaml", "r") as f:
    config = yaml.safe_load(f)

config

estimators = build_estimators(config)

In [32]:
from src.utils import load_titanic_data

pipeline = estimators["ml_pipe"]
X, y, _ = load_titanic_data()

pipeline.fit(X, y)

Pipeline(steps=[('data_prep',
                 DynamicDataPrepPipeline(age_imputer_model=RandomForestRegressor(max_depth=20),
                                         extract_deck=True, extract_fam=True,
                                         extract_sexpclassage=True,
                                         extract_title=True,
                                         numeric_columns=['Age', 'Pclass',
                                                          'Fare'],
                                         numeric_transformations={'Fare': Pipeline(steps=[('bin',
                                                                                           KBinsDiscretizer(encode='ordinal',
                                                                                                            n_bins=4)),
                                                                                          ('scale',
                                                                                           MinMaxScaler())]),
                                                                  'Pclass': MinMaxScaler()},
                                         ordinal_columns=['Sex'])),
                ('model',
                 RandomForestClassifier(max_depth=20, random_state=0))])

In [36]:
pipeline.named_steps["data_prep"].pipeline_

Pipeline(steps=[('extractor',
                 Pipeline(steps=[('fare_imputer', FareImputer()),
                                 ('fam', FamilySizeExtractor()),
                                 ('title', TitleExtractor()),
                                 ('deck', DeckExtractor()),
                                 ('age_imputer',
                                  AgeImputer(model=RandomForestRegressor(max_depth=20))),
                                 ('sexpclassage', SexPclassAgeExtractor())])),
                ('col_tf',
                 ColumnTransformer(transformers=[('num_Fare',
                                                  Pipeline(steps=[('bin',
                                                                   K...
                                                                   OrdinalEncoder(categories=[['U',
                                                                                               'G',
                                                                                               'F',
                                                                                               'E',
                                                                                               'D',
                                                                                               'C',
                                                                                               'B',
                                                                                               'A']])),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  ['Deck']),
                                                 ('onehot_Sex',
                                                  OneHotEncoder(categories=[['male',
                                                                             'female']]),
                                                  ['Sex']),
                                                 ('onehot_Title',
                                                  OneHotEncoder(categories=[['Mr',
                                                                             'Miss',
                                                                             'Mrs',
                                                                             'Master',
                                                                             'Dr',
                                                                             'Rev',
                                                                             'Unknown']],
                                                                handle_unknown='ignore'),
                                                  ['Title']),
                                                 ('onehot_SexPclassAge',
                                                  OneHotEncoder(),
                                                  ['SexPclassAge'])]))])

In [15]:
import json, yaml

with open("config.json", "w") as f:
    json.dump(config, fp=f)
# print(yaml.safe_dump(config))

In [17]:
with open("config.json", "r") as f:
    configload = json.load(f)

configload

{'experiment_name': '000_example',
 'data_prep': {'type': 'DynamicDataPrepPipeline',
  'params': {'ordinal_columns': ['Sex'],
   'numeric_columns': ['Age', 'Pclass', 'Fare'],
   'extract_title': True,
   'extract_fam': True,
   'extract_deck': True,
   'extract_sexpclassage': True}},
 'model': {'type': 'RandomForestClassifier',
  'params': {'max_depth': 20, 'random_state': 0}},
 'splitter': {'type': 'StratifiedKFold',
  'params': {'n_splits': 10, 'shuffle': True, 'random_state': 0}},
 'numeric_transformations': {'Fare': {'type': 'Pipeline',
   'steps': [{'name': 'bin',
     'transformer': {'type': 'KBinsDiscretizer',
      'params': {'n_bins': 4, 'encode': 'ordinal'}}},
    {'name': 'scale', 'transformer': {'type': 'MinMaxScaler'}}]},
  'Pclass': {'type': 'MinMaxScaler'}},
 'age_imputer_model': {'type': 'RandomForestRegressor',
  'params': {'max_depth': 20}}}